In [1]:
import argparse
import os

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from importlib import import_module
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
data_dir='/opt/ml/input/data'
dataset='CustomDataLoader'
num_classes=12
test_name='test.json'
num_workers=2
batch_size=64
test_augmentation='TestAugmentation'

model0_dir='./model/DeepLabV3Plus_ResNet152_Adamp_0.0001_cutout30_plus_pseudo_cos_softcrossentropyloss0.5_all'
model1_dir='./model/DeepLabV3Plus_SeResNext101_Adamp_0.0001_cutout30_plus_pseudo_cos_softcrossentropyloss0.52'


output_name='softensemblebest.csv'

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


In [4]:
test_path=os.path.join(data_dir,test_name)
dataset_module=getattr(import_module('dataset'),dataset)   # default : CustomDataLoader
test_transform_module = getattr(import_module("dataset"),test_augmentation)  # default: TestAugmentation
test_transform=test_transform_module()  # augmentation 에 입력값을 넣어주고 싶으면 TestAugmentation을 수정하고 여기에 넣어주면된다.
# test dataset
test_dataset = dataset_module(data_dir=test_path, mode='test', transform=test_transform)

def collate_fn(batch):
    return tuple(zip(*batch))

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
#                                             num_workers=num_workers,
                                            collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/opt/ml/realcode/dataset.py:112: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  new_data=np.array(new_data)


In [5]:
for i in range(2):
    if i==0:
        model_dir=model0_dir
        model=smp.DeepLabV3Plus(encoder_name='resnet152',classes=num_classes,encoder_weights='imagenet',activation=None)
        
    elif i==1:
        model_dir=model1_dir
        model=smp.DeepLabV3Plus(encoder_name='se_resnext101_32x4d',classes=num_classes,encoder_weights='imagenet',activation=None)
#     elif i==2:
#         model_dir=model2_dir
#     elif i==3:
#         model_dir=model3_dir
#     elif i==4:
#         model_dir=model4_dir
    model_path = os.path.join(model_dir, 'best.pth')
    model.load_state_dict(torch.load(model_path,map_location=device))
    model.to(device)


#     test_path=os.path.join(data_dir,test_name)
#     dataset_module=getattr(import_module('dataset'),dataset)   # default : CustomDataLoader
#     test_transform_module = getattr(import_module("dataset"),test_augmentation)  # default: TestAugmentation
#     test_transform=test_transform_module()  # augmentation 에 입력값을 넣어주고 싶으면 TestAugmentation을 수정하고 여기에 넣어주면된다.
#     # test dataset
#     test_dataset = dataset_module(data_dir=test_path, mode='test', transform=test_transform)

#     def collate_fn(batch):
#         return tuple(zip(*batch))

#     test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                             batch_size=batch_size,
# #                                             num_workers=num_workers,
#                                             collate_fn=collate_fn)
    submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()

    file_name_list = []
    preds_array1 = np.empty((0, 12,512*512), dtype=np.long)
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(test_loader):

            # (batch,channel,512,512)
            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            # # (batch,512,512)
            # oms = torch.argmax(outs, dim=1).detach().cpu().numpy()

            # # resize (256 x 256)
            # temp_mask = []
            # for img, mask in zip(np.stack(imgs), oms):
            #     transformed = transform(image=img, mask=mask)
            #     # (256,256)
            #     mask = transformed['mask']
            #     temp_mask.append(mask)
            #(batch,256,256)

            # (batch,12,512,512)
            oms=outs.detach().cpu().numpy()
            #(batch,12,512*512)
            oms = oms.reshape([oms.shape[0],oms.shape[1],512*512]).astype(int)
            preds_array1 = np.vstack((preds_array1, oms))
            # 배치가 쌓이다가 (배치+배치+배치+...,12,256*256) 
            # (전체데이터개수,12,256*256) 와 같이 된다.

            # i는 batch 중에서 한개 그 한개의 파일이름을 배치만큼모아놓은리스트[f1,f2,f3,..fbatch]를 file_name_list에 넣는다.
            file_name_list.append([i['file_name'] for i in image_infos])
    
    if i==0:
        preds_array=preds_array1
    else:
        preds_array+=preds_array1
    print(preds_array.shape)

Start prediction.
(837, 12, 262144)
Start prediction.
(837, 12, 262144)


In [6]:
print(preds_array.shape)

(837, 12, 262144)


In [7]:
preds_array1=preds_array/2 # 2로 나누는거는 2개 앙상블이므로므로
preds_array1=np.argmax(preds_array1,1)
print(preds_array1.shape)
preds_array1=preds_array1.reshape([preds_array1.shape[0],512,512]).astype(int)
transform = A.Compose([A.Resize(256, 256)])


(837, 262144)


In [8]:
preds_array1=preds_array1.reshape([837,512,512]).astype(int)
print(preds_array1.shape)
images=np.random.rand(837,3,512,512)
print(images.shape)
preds_array2=[]
for images,masks in zip(images,preds_array1):
    transformed = transform(image=images,mask=masks)
    preds_array2.append(transformed['mask'])
preds_array1=np.array(preds_array2)
print(preds_array1)

(837, 512, 512)
(837, 3, 512, 512)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 3 3 3]
  [0 0 0 ... 3 3 3]
  [0 0 0 ... 3 3 3]]]


In [9]:
print(preds_array1.shape)
preds_array1=preds_array1.reshape([837,256*256]).astype(int)
print(preds_array1.shape)

(837, 256, 256)
(837, 65536)


In [10]:
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

In [11]:
data0=pd.read_csv('/opt/ml/realcode/output/DeepLabV3Plus_ResNet152_Adamp_0.0001_marginx_plus_plus_notb_pseudo_cos_kfold0.csv')
file_names=data0['image_id']
for file_name, string in zip(file_names, preds_array1):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                ignore_index=True)
submission.to_csv(os.path.join('./output','softensemble.csv'), index=False)